In [ ]:
import pickle
import gzip
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import random

import cv2
import albumentations as A

import torchvision.transforms as T
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from google.colab import drive
drive.mount('/content/drive')
PATH = "drive/My Drive/data/aml_task_3"

In [ ]:
# helper functions --------------
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object


def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)


def show_side_by_side(video, mask):
    fig = plt.figure()
    fig.add_subplot(1, 2, 1)
    plt.imshow(video)
    fig.add_subplot(1, 2, 2)
    plt.imshow(mask)

In [ ]:
def ensem_pred(predictions):
    """ensemble predictions from a list of images"""
    ensemble_pred = None
    for pred in predictions:
        if ensemble_pred is None:
            ensemble_pred = torch.clone(pred).float()  # Copy here!!
        else:
            ensemble_pred += pred.float()
    ensemble_pred = ensemble_pred / len(predictions)
    return ensemble_pred

In [ ]:
model_predictions = [load_zipped_pickle(f"{PATH}/model_{i}.pkl") for i in range(1,5)]
data_test = load_zipped_pickle("{}/test.pkl".format(PATH))

submission = []
for vid_idx, item in enumerate(data_test):
    name = item["name"]
    video = item["video"]
    shape = video[:, :, 0].shape
    resizer = A.Resize(shape[0], shape[1])
    predictions = []
    for idx in range(video.shape[2]):
        # ensemble here
        frame_predictions = [pred[vid_idx][idx] for pred in model_predictions]
        prediction = ensem_pred(frame_predictions)
        prediction = prediction.cpu().detach().numpy().squeeze()
        prediction = prediction > 0.3
        predictions.append(prediction)
    predictions = np.stack(predictions, axis=2)
    submission.append({"name":name, "prediction":predictions})

save_zipped_pickle(submission, "{}/ensemble_prediction.pkl".format(PATH))